# Установка и загрузка модели

#### HF_TOKEN - Секрет в Google Colab, его нужно будет добавить самостоятельно

In [1]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from google.colab import userdata
from huggingface_hub import login
login(token=userdata.get('HF_TOKEN'))

Выбрать модель: [Hugging Face](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d)

In [3]:
from unsloth import FastLanguageModel


max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = userdata.get('HF_TOKEN'),
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.9: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
# Настройка LoRA адаптера
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,


)

Unsloth 2025.2.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Загрузка и форматирование датасета



Так как DeepSeek-R1-Distill-Llama-8B не является Thinking на Non-English языках, попробуем дообучить ее на longCoT Russian датасете, подробнее:
[Hugging Face](https://huggingface.co/datasets/kristaller486/Nebo-T1-Russian)

In [5]:
train_prompt_style = """Ниже приведена инструкция, описывающая задание, в паре с вводными данными, которые обеспечивают дополнительный контекст.
Напишите ответ, который бы соответствовал запросу.
Прежде чем отвечать, тщательно обдумайте вопрос и выстройте пошаговую цепочку мыслей, чтобы ответ был логичным и точным.

### Instruction:
Вы являетесь экспертом с продвинутыми знаниями в области математических рассуждений, логики, письма.
Пожалуйста, ответьте на следующий вопрос.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [6]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    inputs = examples["prompt"]
    cots = examples["think"]
    outputs = examples["answer"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

Датасет должен содержать "Thinking" часть, например: [Dataset (EN)](https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/en/train) или [Dataset (RU)](https://huggingface.co/datasets/kristaller486/Nebo-T1-Russian)

In [7]:
from datasets import load_dataset

dataset = load_dataset("kristaller486/Nebo-T1-Russian", split = "train", trust_remote_code=True)

dataset = dataset.filter(lambda x: x["russian_only"])

dataset = dataset.map(formatting_prompts_func, batched = True)

In [8]:
dataset["text"][0]

'Ниже приведена инструкция, описывающая задание, в паре с вводными данными, которые обеспечивают дополнительный контекст.\nНапишите ответ, который бы соответствовал запросу.\nПрежде чем отвечать, тщательно обдумайте вопрос и выстройте пошаговую цепочку мыслей, чтобы ответ был логичным и точным.\n\n### Instruction:\nВы являетесь экспертом с продвинутыми знаниями в области математических рассуждений, логики, письма.\nПожалуйста, ответьте на следующий вопрос.\n\n### Question:\nКакова критическая температура сверхпроводящей тонкой пленки, изготовленной из свинца толщиной 100 нм? И как критическая температура меняется с толщиной пленки свинца, предполагая, что пленка идеально чистая и свободна от любых дефектов?\n\n### Response:\n<think>\nХорошо, давайте разберемся с этим вопросом о критической температуре сверхпроводящей тонкой пленки свинца. Начнем с того, что я помню из курса физики твердого тела. Критическая температура (Tc) – это температура, ниже которой материал переходит в сверхпров

# Устанвока гиперпараметров для обучения

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to = "none",
    ),
)

# Обучение модели

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 12,740 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.070400
20,0.794100
30,0.754900
40,0.734900
50,0.716400
60,0.748000


# Тестирование

In [11]:
prompt_style = """Ниже приведена инструкция, описывающая задание, в паре с вводными данными, которые обеспечивают дополнительный контекст.
Напишите ответ, который бы соответствовал запросу.
Прежде чем отвечать, тщательно обдумайте вопрос и выстройте пошаговую цепочку мыслей, чтобы ответ был логичным и точным.

### Instruction:
Вы являетесь экспертом с продвинутыми знаниями в области математических рассуждений, логики, письма.
Пожалуйста, ответьте на следующий вопрос.

### Question:
{}

### Response:
<think>{}"""

In [12]:
question = "Реши интеграл ∫(2x^2+3)dx, в границах интегрирования от 0 до 9"


FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Хорошо, давайте разбираться с интегралом ∫(2x² + 3)dx от 0 до 9. Сначала вспомню основы интегрирования. Интеграл от 0 до 9 – это не просто вычисление функции в 9 и 0 и умножение на разность, но именно вычисление разности функций в этих точках. Поскольку функция 2x² + 3 является полиномом, её интеграл должен быть несложным. 

Начну с разложения на члены. Интеграл 2x² + 3 можно разбить на две части: ∫2x²dx + ∫3dx. Это потому, что интеграл суммы равен сумме интегралов. Теперь нужно найти интеграл каждого члена отдельно.

Для первого члена, 2x²dx. Поскольку это квадратный член, его интеграл будет найден с помощью формулы ∫x^n dx = (x^(n+1))/(n+1) + C. Здесь n = 2, поэтому интеграл 2x²dx будет 2*(x³/3) + C, то есть (2/3)x³ + C. Но здесь C – это константа интегрирования, которую мы не учитываем при вычислении конкретного интеграла от 0 до 9.

Второй член – 3dx. Интеграл этого будет 3x + C. Так что, объединяя оба члена, полный интеграл функции будет (2/3)x³ + 3x + C.

Теперь нужно вы

In [13]:
response[0]

'<｜begin▁of▁sentence｜>Ниже приведена инструкция, описывающая задание, в паре с вводными данными, которые обеспечивают дополнительный контекст.\nНапишите ответ, который бы соответствовал запросу.\nПрежде чем отвечать, тщательно обдумайте вопрос и выстройте пошаговую цепочку мыслей, чтобы ответ был логичным и точным.\n\n### Instruction:\nВы являетесь экспертом с продвинутыми знаниями в области математических рассуждений, логики, письма.\nПожалуйста, ответьте на следующий вопрос.\n\n### Question:\nРеши интеграл ∫(2x^2+3)dx, в границах интегрирования от 0 до 9\n\n### Response:\n<think>\nХорошо, давайте разбираться с интегралом ∫(2x² + 3)dx от 0 до 9. Сначала вспомню основы интегрирования. Интеграл от 0 до 9 – это не просто вычисление функции в 9 и 0 и умножение на разность, но именно вычисление разности функций в этих точках. Поскольку функция 2x² + 3 является полиномом, её интеграл должен быть несложным. \n\nНачну с разложения на члены. Интеграл 2x² + 3 можно разбить на две части: ∫2x²dx 

# Сохранение модели
https://huggingface.co/settings/tokens - Получение токена (Нужно выдать all grants)

Вместо *hf* - свое имя на Hugging Face, *model* - имя репозитория

In [ ]:
# Сохранить в 16bit (Исходное качество)
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Сохранить в 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit_forced",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit_forced", token = "")

# Только LoRA адаптер
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")